# Оценка способности потенциального заёмщика вернуть кредит банку

# Проект

## Описание проекта

Кредитный отдел банка, нужно ответить на вопрос: влияет ли количество детей и семейное положение на погашение кредита. Построить модель, которая оценивает способность клиента вернуть кредит.

## Цель проекта

Ответить на следующие вопросы:

 - Есть ли зависимость между наличием детей и возвратом кредита в срок?

 - Есть ли зависимость между семейным положением и возвратом кредита в срок?

 - Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

 - Как разные цели кредита влияют на его возврат в срок?


## План проекта

 -  Знакомство с данными
 -  Предобработка данных
 -  Анализ данных
 -  Общий вывод

# Загрузка и изучение данных

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('/datasets/data.csv')

In [3]:
df


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,-4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791.862382,операции с жильем
21521,0,343937.404131,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999.806512,сделка с автомобилем
21522,1,-2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672.561153,недвижимость
21523,3,-3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093.050500,на покупку своего автомобиля


In [4]:
df['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

Видим, что данные не совсем корректны. -1 ребенок в семье быть не может, поэтому - нужно убрать(скорее всего имелось ввиду, что в семье 1 ребенок).



In [5]:
df['days_employed']

0         -8437.673028
1         -4024.803754
2         -5623.422610
3         -4124.747207
4        340266.072047
             ...      
21520     -4529.316663
21521    343937.404131
21522     -2113.346888
21523     -3112.481705
21524     -1984.507589
Name: days_employed, Length: 21525, dtype: float64

Есть значения, которые слишком большие. Например, в строке 4 340266 дней, это 932 года. Нужно это исправлять.

In [6]:
df['days_employed'].isna().sum()

2174

Пропущенных значений 2174

Из первых строк видно, что данные записаны некорректны, т.к трудовой стаж не может быть
отрицательным, нужно минус убрать


In [7]:
df['dob_years'].value_counts()

35    617
40    609
41    607
34    603
38    598
42    597
33    581
39    573
31    560
36    555
44    547
29    545
30    540
48    538
37    537
50    514
43    513
32    510
49    508
28    503
45    497
27    493
56    487
52    484
47    480
54    479
46    475
58    461
57    460
53    459
51    448
59    444
55    443
26    408
60    377
25    357
61    355
62    352
63    269
64    265
24    264
23    254
65    194
66    183
22    183
67    167
21    111
0     101
68     99
69     85
70     65
71     58
20     51
72     33
19     14
73      8
74      6
75      1
Name: dob_years, dtype: int64

Видим, что данные некорректны, возраст не может быть 0

In [8]:
df['education'].unique()

array(['высшее', 'среднее', 'Среднее', 'СРЕДНЕЕ', 'ВЫСШЕЕ',
       'неоконченное высшее', 'начальное', 'Высшее',
       'НЕОКОНЧЕННОЕ ВЫСШЕЕ', 'Неоконченное высшее', 'НАЧАЛЬНОЕ',
       'Начальное', 'Ученая степень', 'УЧЕНАЯ СТЕПЕНЬ', 'ученая степень'],
      dtype=object)

Данные записаны по-разному, в разных регистрах. Нужно сделать все в одном регистре.

In [9]:
df['education_id'].value_counts()

1    15233
0     5260
2      744
3      282
4        6
Name: education_id, dtype: int64

Делаем вывод, что:
    - 0 - высшее образование
    - 1 - среднее образование
    - 2 - неоконченное высшее
    - 3 - начальное 
    - 4 - ученая степень

In [10]:
df['family_status'].value_counts()

женат / замужем          12380
гражданский брак          4177
Не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64

In [11]:
df['family_status_id'].value_counts()

0    12380
1     4177
4     2813
3     1195
2      960
Name: family_status_id, dtype: int64

Делаем вывод, что:
    - 0 - женат / замужем
    - 1 - гражданский брак        
    - 2 - вдовец / вдова     
    - 3 - в разводе                 
    - 4 - Не женат / не замужем 
   
    

In [12]:
df['gender'].value_counts()

F      14236
M       7288
XNA        1
Name: gender, dtype: int64

Делаем вывод, что одно значение неверное

In [13]:
df['income_type'].value_counts()

сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
безработный            2
предприниматель        2
студент                1
в декрете              1
Name: income_type, dtype: int64

Здесь все записано корректно

In [14]:
df['debt'].value_counts()

0    19784
1     1741
Name: debt, dtype: int64

Видим, что задолженности имеют не значительное количество людей

In [15]:
df.total_income

0        253875.639453
1        112080.014102
2        145885.952297
3        267628.550329
4        158616.077870
             ...      
21520    224791.862382
21521    155999.806512
21522     89672.561153
21523    244093.050500
21524     82047.418899
Name: total_income, Length: 21525, dtype: float64

In [16]:
df['purpose'].value_counts()

свадьба                                   797
на проведение свадьбы                     777
сыграть свадьбу                           774
операции с недвижимостью                  676
покупка коммерческой недвижимости         664
покупка жилья для сдачи                   653
операции с жильем                         653
операции с коммерческой недвижимостью     651
покупка жилья                             647
жилье                                     647
покупка жилья для семьи                   641
строительство собственной недвижимости    635
недвижимость                              634
операции со своей недвижимостью           630
строительство жилой недвижимости          626
покупка недвижимости                      624
покупка своего жилья                      620
строительство недвижимости                620
ремонт жилью                              612
покупка жилой недвижимости                607
на покупку своего автомобиля              505
заняться высшим образованием      

Видим, что категорий много на первый взгляд. Посмотрим что будет после удаления дубликатов

# Предобработка данных

## Устранение ошибок в данных

Ошибки в children

In [17]:
df['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

Предположим, что -1 ребенок в семье, это имелось ввиду 1 ребенок в семье, поэтому возьмем количество детей по модулю

In [18]:
df['children'] = abs(df['children'])

In [19]:
df['children'].value_counts()

0     14149
1      4865
2      2055
3       330
20       76
4        41
5         9
Name: children, dtype: int64

Ошибки в days_employed

Необходимо убрать минусы

In [20]:
df['days_employed'] = abs(df['days_employed'])

Минусы убрали

Ошибки в dob_years

Необходимо заменить 0 на nan

In [21]:
df['dob_years'].replace(0,np.nan,inplace=True)

Считаем медиану возраста клиента, чтобы потом заменить nan на медиану

In [22]:
median_dob_years = df['dob_years'].median()

In [23]:
df['dob_years'] = df['dob_years'].fillna(median_dob_years)

### Вывод по ошибкам

Проверяем, что все ошибки устранены.

In [24]:
df.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42.0,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36.0,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33.0,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32.0,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53.0,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,926.185831,27.0,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,2879.202052,43.0,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,152.779569,50.0,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,6929.865299,35.0,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,2188.756445,41.0,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


## Обработка пропущенных значений


In [25]:
df.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Видим, что пропущенные значения только в days_employed и в total_income

Изменение регистра в education

Меняем регистр для удобства.

In [26]:
df['education'] = df['education'].str.lower()

Пропущенные значения в days_employed

1. Определяем сколько приходится трудового стажа на 1 год 

Создаем новый столбец.
Вычитаем из возраста клиентов 18 лет. До 18 лет не работают.

In [27]:
df['days_employed_per_year'] = df['days_employed'] / (df['dob_years']-18)

Меняем завышенный стаж на nan. 730 - двойной стаж (365*2)

In [28]:
df.loc[df.days_employed_per_year > 730,'days_employed_per_year'] = np.nan

Ищем медиану по столбцу days_employed_per_year.

In [29]:
median_days_employed_per_year = df['days_employed_per_year'].median()

2. Пишем функцию, чтобы nan в столбце days_employed_per_year менялись на произведение медианы и возраста.

In [30]:
def fill_days_per_year(row):
    if (np.isnan(row['days_employed_per_year'])):
      return median_days_employed_per_year * row['dob_years']
    else:
        return row['days_employed']
        
df['days_employed'] = df.apply(fill_days_per_year,axis = 1)

Убеждаемся, что все сработало правильно. Видим, что теперь завышенного стажа нет

In [31]:
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,days_employed_per_year
0,1,8437.673028,42.0,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья,351.569709
1,1,4024.803754,36.0,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля,223.600209
2,0,5623.422610,33.0,среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья,374.894841
3,3,4124.747207,32.0,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование,294.624800
4,0,4733.371543,53.0,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу,NaN


Пропущенные значения в total_income

In [32]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 13 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   children                21525 non-null  int64  
 1   days_employed           21525 non-null  float64
 2   dob_years               21525 non-null  float64
 3   education               21525 non-null  object 
 4   education_id            21525 non-null  int64  
 5   family_status           21525 non-null  object 
 6   family_status_id        21525 non-null  int64  
 7   gender                  21525 non-null  object 
 8   income_type             21525 non-null  object 
 9   debt                    21525 non-null  int64  
 10  total_income            19351 non-null  float64
 11  purpose                 21525 non-null  object 
 12  days_employed_per_year  15899 non-null  float64
dtypes: float64(4), int64(4), object(5)
memory usage: 2.1+ MB


1. Определяем медиану ежемесячного дохода относительно образования 

Медиана для высшего образования

In [33]:
income_higher = df.loc[df['education']=='высшее']['total_income'].median()

Медиана для среднего образования

In [34]:
income_secondary = df.loc[df['education']=='среднее']['total_income'].median()

Медиана для неоконченного высшего образования

In [35]:
income_uncompleted_higher  = df.loc[df['education']=='неоконченное высшее']['total_income'].median()

Медиана для начального образования

In [36]:
income_primary  = df.loc[df['education']=='начальное']['total_income'].median()

Медиана для ученой степени

In [37]:
income_degree  = df.loc[df['education']=='ученая степень']['total_income'].median()

2. Обновялем df. Заменяем пропущенные значения в total_income медианой.

In [38]:
df.loc[df['education'] == 'высшее', 'total_income'] = df.loc[df['education'] == 'высшее', 'total_income'].fillna(income_higher)

In [39]:
df.loc[df['education'] == 'среднее', 'total_income'] = df.loc[df['education'] == 'среднее', 'total_income'].fillna(income_secondary)

In [40]:
df.loc[df['education'] == 'неоконченное высшее', 'total_income'] = df.loc[df['education'] == 'неоконченное высшее', 'total_income'].fillna(income_uncompleted_higher)

In [41]:
df.loc[df['education'] == 'начальное', 'total_income'] = df.loc[df['education'] == 'начальное', 'total_income'].fillna(income_primary)

In [42]:
df.loc[df['education'] == 'ученая степень', 'total_income'] = df.loc[df['education'] == 'ученая степень', 'total_income'].fillna(income_degree)

### Вывод по пропущенным значениям

Убеждаемся, что пропущенных значений нет.

In [43]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 13 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   children                21525 non-null  int64  
 1   days_employed           21525 non-null  float64
 2   dob_years               21525 non-null  float64
 3   education               21525 non-null  object 
 4   education_id            21525 non-null  int64  
 5   family_status           21525 non-null  object 
 6   family_status_id        21525 non-null  int64  
 7   gender                  21525 non-null  object 
 8   income_type             21525 non-null  object 
 9   debt                    21525 non-null  int64  
 10  total_income            21525 non-null  float64
 11  purpose                 21525 non-null  object 
 12  days_employed_per_year  15899 non-null  float64
dtypes: float64(4), int64(4), object(5)
memory usage: 2.1+ MB


Нужно удалить ненужный столбец days_employed_per_year

In [44]:
del df['days_employed_per_year']

In [45]:
df

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42.0,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36.0,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33.0,среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32.0,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,4733.371543,53.0,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,4529.316663,43.0,среднее,1,гражданский брак,1,F,компаньон,0,224791.862382,операции с жильем
21521,0,5983.696102,67.0,среднее,1,женат / замужем,0,F,пенсионер,0,155999.806512,сделка с автомобилем
21522,1,2113.346888,38.0,среднее,1,гражданский брак,1,M,сотрудник,1,89672.561153,недвижимость
21523,3,3112.481705,38.0,среднее,1,женат / замужем,0,M,сотрудник,1,244093.050500,на покупку своего автомобиля


## Замена типов данных


In [46]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     21525 non-null  float64
 2   dob_years         21525 non-null  float64
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(3), int64(4), object(5)
memory usage: 2.0+ MB


Необходимо заменить тип данных float64 на int64

In [47]:
df['days_employed'] = df['days_employed'].astype('int')

In [48]:
df['dob_years'] = df['dob_years'].astype('int')

In [49]:
df['total_income'] = df['total_income'].astype('int')

Использовал метод astype потому, что мне он кажется здесь уместным

### Вывод

In [50]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          21525 non-null  int64 
 1   days_employed     21525 non-null  int64 
 2   dob_years         21525 non-null  int64 
 3   education         21525 non-null  object
 4   education_id      21525 non-null  int64 
 5   family_status     21525 non-null  object
 6   family_status_id  21525 non-null  int64 
 7   gender            21525 non-null  object
 8   income_type       21525 non-null  object
 9   debt              21525 non-null  int64 
 10  total_income      21525 non-null  int64 
 11  purpose           21525 non-null  object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


Видим, что все типы данных float64 стали int64

## Выделение лемм

Выделение лемм в столбце purpose

In [51]:
from pymystem3 import Mystem
m = Mystem() 

In [52]:
df['purpose_lemmas'] = df['purpose'].apply(m.lemmatize)

In [53]:
df['purpose'].value_counts()

свадьба                                   797
на проведение свадьбы                     777
сыграть свадьбу                           774
операции с недвижимостью                  676
покупка коммерческой недвижимости         664
покупка жилья для сдачи                   653
операции с жильем                         653
операции с коммерческой недвижимостью     651
покупка жилья                             647
жилье                                     647
покупка жилья для семьи                   641
строительство собственной недвижимости    635
недвижимость                              634
операции со своей недвижимостью           630
строительство жилой недвижимости          626
покупка недвижимости                      624
покупка своего жилья                      620
строительство недвижимости                620
ремонт жилью                              612
покупка жилой недвижимости                607
на покупку своего автомобиля              505
заняться высшим образованием      

In [54]:
def purpose_change(row):
    if ('свадьба') in row:
        return 'свадьба'
    if ('образование') in row:
        return 'образование'
    if ('автомобиль') in row:
        return 'автомобиль'
    if ('жилье') in row:
        return 'жилье'
    if ('недвижимость') in row:
        return 'недвижимость'
df['purpose_credit'] = df.purpose_lemmas.apply(purpose_change)

In [55]:
df['purpose_credit'].value_counts()

недвижимость    6367
жилье           4473
автомобиль      4315
образование     4022
свадьба         2348
Name: purpose_credit, dtype: int64

### Вывод: 

Теперь дубликатов нет. Оказалось, что самая популярная цель - это кредит на недвижимость.

## Категоризация данных

Создаем новый датафрейм для образования

In [56]:
df_education = df[['education','education_id']]

In [57]:
df_education = df_education.drop_duplicates().reset_index(drop=True)

In [58]:
df_education

,education,education_id
0,высшее,0
1,среднее,1
2,неоконченное высшее,2
3,начальное,3
4,ученая степень,4


In [59]:
del df ['education']

Теперь у нас есть новая таблица 

Делаем то же самое, что и с образованием

In [60]:
df_family_status = df[['family_status','family_status_id']]

In [61]:
df_family_status = df_family_status.drop_duplicates().reset_index(drop=True)

In [62]:
df_family_status

,family_status,family_status_id
0,женат / замужем,0
1,гражданский брак,1
2,вдовец / вдова,2
3,в разводе,3
4,Не женат / не замужем,4


Еще одна новая таблица

In [63]:
del df['family_status']

In [64]:
df

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,purpose_lemmas,purpose_credit
0,1,8437,42,0,0,F,сотрудник,0,253875,покупка жилья,"[покупка, , жилье, \n]",жилье
1,1,4024,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,"[приобретение, , автомобиль, \n]",автомобиль
2,0,5623,33,1,0,M,сотрудник,0,145885,покупка жилья,"[покупка, , жилье, \n]",жилье
3,3,4124,32,1,0,M,сотрудник,0,267628,дополнительное образование,"[дополнительный, , образование, \n]",образование
4,0,4733,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,"[сыграть, , свадьба, \n]",свадьба
...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,4529,43,1,1,F,компаньон,0,224791,операции с жильем,"[операция, , с, , жилье, \n]",жилье
21521,0,5983,67,1,0,F,пенсионер,0,155999,сделка с автомобилем,"[сделка, , с, , автомобиль, \n]",автомобиль
21522,1,2113,38,1,1,M,сотрудник,1,89672,недвижимость,"[недвижимость, \n]",недвижимость
21523,3,3112,38,1,0,M,сотрудник,1,244093,на покупку своего автомобиля,"[на, , покупка, , свой, , автомобиль, \n]",автомобиль


### Вывод:

In [65]:
del df['purpose']

In [66]:
del df['purpose_lemmas']

In [67]:
df

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose_credit
0,1,8437,42,0,0,F,сотрудник,0,253875,жилье
1,1,4024,36,1,0,F,сотрудник,0,112080,автомобиль
2,0,5623,33,1,0,M,сотрудник,0,145885,жилье
3,3,4124,32,1,0,M,сотрудник,0,267628,образование
4,0,4733,53,1,1,F,пенсионер,0,158616,свадьба
...,...,...,...,...,...,...,...,...,...,...
21520,1,4529,43,1,1,F,компаньон,0,224791,жилье
21521,0,5983,67,1,0,F,пенсионер,0,155999,автомобиль
21522,1,2113,38,1,1,M,сотрудник,1,89672,недвижимость
21523,3,3112,38,1,0,M,сотрудник,1,244093,автомобиль


# Анализ данных

## Определение зависимости между наличием детей и возвратом кредита в срок

In [68]:
df[df['children']  == 0]['debt'].mean()

0.0751289843805216

Процент неплательщиков без детей составляет 7,5%

In [69]:
df[df['children']  > 0]['debt'].mean()

0.09191973969631237

Процент неплательщиков с детьми составляет 9,2% 

### Вывод:  

Наличие детей влияет на возрат кредита в срок.

## Определение зависимости между семейным положением и возвратом кредита в срок

1.Посчитаем процент неплательщиков женатых/замужем

In [70]:
df[df['family_status_id']  == 0]['debt'].mean()

0.07520193861066236

Процент неплательщиков женатых/замужем составляет 7,5%

2.Посчитаем процент неплательщиков в гражданском браке

In [71]:
df[df['family_status_id']  == 1]['debt'].mean()

0.09288963370840315

Процент неплательщиков в гражданском браке составляет 9,2%

3.Посчитаем процент неплательщиков вдовцов/вдов

In [72]:
df[df['family_status_id']  == 2]['debt'].mean()

0.065625

Процент неплательщиков вдовцов/вдов составляет 6,5%

4.Посчитаем процент неплательщиков в разводе

In [73]:
df[df['family_status_id']  == 3]['debt'].mean()

0.07112970711297072

Процент неплательщиков в разводе составляет 7,1%

4.Посчитаем процент неплательщиков не женатых/не замужем

In [74]:
df[df['family_status_id']  == 4]['debt'].mean()

0.09740490579452542

Процент неплательщиков не женатых/не замужем составляет 9,7%

### Вывод: 

Хуже всего возвращают не женатые или не замужем люди. Лучше всего возвращают люди в равзоде. Т.е мы можем сделать вывод: Возвращение кредита в срок зависит от семейного положения.

## Определение зависимости между уровнем дохода и возвратом кредита в срок

Разделим уровень дохода на категории:
-  Высокий уровень дохода - выше 150 000 руб
-  Средний уровень дохода - от 60 000 до 150 000 руб
-  Низкий уровень дохода - до 60 000 руб.

In [75]:
df[df['total_income'] < 60000]['debt'].mean()

0.060794044665012405

In [76]:
df.loc[(df['total_income'] > 60000) & (df['total_income'] < 150000)]['debt'].mean()

0.0857901483244827

In [77]:
df[df['total_income'] > 150000]['debt'].mean()

0.077064407471675

### Вывод: 

Видим, что возврат кредита в срок зависит от уровня дохода, люди с средним уровнем дохода возращают кредиты хуже, чем люди остальные.

## Определение зависимости между целью кредита и возратом кредита в срок

In [78]:
df['purpose_credit'].value_counts()

недвижимость    6367
жилье           4473
автомобиль      4315
образование     4022
свадьба         2348
Name: purpose_credit, dtype: int64

In [79]:
df.groupby('purpose_credit')['debt'].value_counts()

purpose_credit  debt
автомобиль      0       3912
                1        403
жилье           0       4165
                1        308
недвижимость    0       5893
                1        474
образование     0       3652
                1        370
свадьба         0       2162
                1        186
Name: debt, dtype: int64

Процент неплательщиков с целью кредита - автомобиль

In [80]:
df.loc[(df['purpose_credit'] == 'автомобиль') & (df['debt'] == 1)]['purpose_credit'].count() / df.loc[df['purpose_credit'] == 'автомобиль']['purpose_credit'].count()

0.09339513325608342

Процент неплательщиков с целью кредита - жилье

In [81]:
df.loc[(df['purpose_credit'] == 'жилье') & (df['debt'] == 1)]['purpose_credit'].count() / df.loc[df['purpose_credit'] == 'жилье']['purpose_credit'].count()

0.06885758998435054

Процент неплательщиков с целью кредита - недвижимость

In [82]:
df.loc[(df['purpose_credit'] == 'недвижимость') & (df['debt'] == 1)]['purpose_credit'].count() / df.loc[df['purpose_credit'] == 'недвижимость']['purpose_credit'].count()

0.07444636406470866

Процент неплательщиков с целью кредита - образование

In [83]:
df.loc[(df['purpose_credit'] == 'образование') & (df['debt'] == 1)]['purpose_credit'].count() / df.loc[df['purpose_credit'] == 'образование']['purpose_credit'].count()

0.0919940328194928

Процент неплательщиков с целью кредита - свадьба

In [84]:
df.loc[(df['purpose_credit'] == 'свадьба') & (df['debt'] == 1)]['purpose_credit'].count() / df.loc[df['purpose_credit'] == 'свадьба']['purpose_credit'].count()

0.07921635434412266

### Вывод: 

Зависимость между целью и возратом кредита в срок присутствует. 
1. Автомобиль - 9,3 %
2. Жилье - 6,8 %
3. Недвижимость - 7,4 %
4. Образование - 9,2 %
5. Свадьба - 7,9 %. 

Хуже всего возращают кредит те, кто берет кредит на автомобиль, лучше всего - жилье.

# Общий вывод

Необходимо было разобраться, влияет ли семейное положение и наличием детей клиента на факт погашения кредита в срок. 
В проекте была проделана работа предобработки данных, а именно: 
 - устранение ошибок в данных;
 - обработка пропущенных значений;
 - замена типов данных;
 - выделение лемм;
 - удаление дубликатов;
 - категоризация данных.
 
 Мы разобрались в том, что семейное положение и наличие детей клиента влияет на факт погашения кредита в срок. И ответили на несколько вопросов, что есть зависимости между уровнем дохода/целью кредита и фактом погашения кредита в срок. 
 В общем на основании этих результатов исследования можно строить модель кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.


